In [1]:
# import libraries
from docxtpl import DocxTemplate, Subdoc
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.enum.section import WD_SECTION, WD_ORIENTATION
from docx.enum.style import WD_STYLE_TYPE
from docx.shared import RGBColor, Mm, Pt, Inches
from docx.oxml import OxmlElement, parse_xml
from docx.oxml.ns import qn, nsdecls
from docx.enum.table import WD_ROW_HEIGHT_RULE
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from urllib.parse import urljoin
import json
from datetime import date, datetime as dt
import copy
import re
import html
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

c:\Users\Gabriel O\Desktop\ACL\ACL\Knowledge Base\Python_Reports\report_temp\Lib\site-packages\docxcompose\properties.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
###############################################################################
# fetch data
####################################################################################

## define variables

BEARER_TOKEN = 'a8aaa668acbc06e39a67d6b3b47d05b7dc510d1399de31484adff466ccc43f7f'
org_id = '11534'
domain_url = 'https://apis-us.diligentoneplatform.com'
headers = {'Authorization' : f'Bearer {BEARER_TOKEN}', 'Content-Type': 'application/vnd.api+json'}

In [3]:
# risk matrix

risk_matrix = {
            'Insignificant':{
                              'Rare':{'value':'L(1)', 'color':'06923E'},
                              'Possible':{'value':'L(2l)', 'color':'06923E'},
                              'Probable':{'value':'L(3l)', 'color':'06923E'},
                              'Likely':{'value':'L(4l)', 'color':'06923E'},
                              'Almost certain':{'value':'L(5l)', 'color':'06923E'}
                            },
                    'Minor':{
                              'Rare':{'value':'L(2i)', 'color':'06923E'},
                              'Possible':{'value':'L(4)', 'color':'06923E'},
                              'Probable':{'value':'M(6l)', 'color':'FFC107'},
                              'Likely':{'value':'M(8l)', 'color':'FFC107'},
                              'Almost certain':{'value':'M(10l)', 'color':'FFC107'}
                            },
                 'Moderate':{
                              'Rare':{'value':'L(3i)', 'color':'06923E'},
                              'Possible':{'value':'M(6i)', 'color':'FFC107'},
                              'Probable':{'value':'M(9)', 'color':'FFC107'},
                              'Likely':{'value':'H(12l)', 'color':'5F8B4C'},
                              'Almost certain':{'value':'H(15l)', 'color':'5F8B4C'}
                            },
                    'Major':{
                              'Rare':{'value':'L(4i)', 'color':'06923E'},
                              'Possible':{'value':'M(8i)', 'color':'FFC107'},
                              'Probable':{'value':'H(12i)', 'color':'5F8B4C'},
                              'Likely':{'value':'VH(16)', 'color':'DD0303'},
                              'Almost certain':{'value':'VH(20l)', 'color':'DD0303'}
                            },
                    'Catastrophic':{
                              'Rare':{'value':'L(5i)', 'color':'06923E'},
                              'Possible':{'value':'M(10i)', 'color':'FFC107'},
                              'Probable':{'value':'H(15i)', 'color':'5F8B4C'},
                              'Likely':{'value':'VH(20i)', 'color':'DD0303'},
                              'Almost certain':{'value':'VH(25)', 'color':'DD0303'}
                            }
}

In [4]:
def resource_url(resource,id):
    """
        Return the full formed endpoint for a specied highbond resource

        Args:
            resource(str): the highbond resources from which to pull data
                  id(int): identifier for the resource
        Return:
                 url(str): url for the api endpoint for data extraction
    """
    if resource.lower()=='project':
       return (
              f'https://apis-us.diligentoneplatform.com/v1/orgs/{org_id}/projects/{id}'
              'fields[projects]=name,state,status,created_at,description,background,'
               'management_response,opinion,opinion_description,purpose,scope,start_date,'
               'target_date,project_type,entities'
               )
    if resource.lower()=='objective':
        return (
                f'https://apis-us.diligentoneplatform.com/v1/orgs/{org_id}/projects/{id}/objectives?'
                'fields[objectives]=title,description,reference,division_department,owner,'
                'executive_owner,created_at,updated_at,project,assigned_user,owner_user,'
                'executive_owner_user,custom_attributes,position,risk_control_matrix_id,'
                'walkthrough_summary_id,entities,framework,framework_origin,risk_assurance_data'
               )
    if resource.lower()=='risk':
        return (
                f'https://apis-us.diligentoneplatform.com/v1/orgs/{org_id}/objectives/{id}/risks?'
                'fields[risks]=title,description,risk_id,owner,position,impact,likelihood,'
                'custom_attributes,custom_factors,created_at,updated_at,objective,mitigations,'
                'owner_user,entities,framework_origin,risk_assurance_data'
               )
    if resource.lower()=='control':
        return (
                f'https://apis-us.diligentoneplatform.com/v1/orgs/{org_id}/objectives/{id}/controls?'
                'fields[controls]=title,description,control_id,owner,frequency,control_type,prevent_detect,'
                'method,status,custom_attributes,objective,walkthrough,control_test_plan,control_tests,'
                'mitigations,framework_origin'
               )
    if resource.lower()=='control_test':
        return (
                f'https://apis-us.diligentoneplatform.com/v1/orgs/{org_id}/control_tests?'
                'fields[control_tests]=assignee_name,testing_round_number,not_applicable,'
                'sample_size,testing_results,testing_conclusion,testing_conclusion_status,'
                'created_at,updated_at,control'
               )
    if resource.lower()=='walkthrough':
        return (
                f'https://apis-us.diligentoneplatform.com/v1/orgs/{org_id}/walkthroughs?'
                'fields[walkthroughs]=walkthrough_results,control_design,created_at,updated_at,'
                 'custom_attributes,control'
               )
    if resource.lower()=='issue':
        return (
                f'https://apis-us.diligentoneplatform.com/v1/orgs/{org_id}/issues?'
                'fields[issues]=title,description,owner,recommendation,deficiency_type,severity,'
                'published,identified_at,reference,reference_prefix,risk,scope,'
                'escalation,cause,effect,cost_impact,executive_summary,executive_owner,'
                'project_owner,closed,remediation_status,remediation_plan,remediation_date,'
                'actual_remediation_date,retest_deadline_date,actual_retest_date,'
                'retesting_results_overview,custom_attributes,project,entities,target'
                )
    if resource.lower()=='mitigation':
        return (
                f'https://apis-us.diligentoneplatform.com/v1/orgs/{org_id}/mitigations'
               )
    if resource.lower()=='action':
        return (
                f'https://apis-us.diligentoneplatform.com/v1/orgs/{org_id}/issues/{id}/actions?'
                'fields[actions]=title,include_issue_details,include_remediation_details,description,'
                'due_date,priority,closed,completed_date,status,submitted_on,slug,'
                'custom_attributes,issue'
               )
    if resource.lower()=='compliance_regulation':
        return (
                f'https://apis-us.diligentoneplatform.com/v1/orgs/{org_id}/compliance_regulations'
               )
    if resource.lower()=='compliance_requirement':
        return (
                f'https://apis-us.diligentoneplatform.com/v1/orgs/{org_id}/compliance_regulations/{id}/compliance_requirements?'
                'fields[compliance_requirements]=applicable,breadcrumbs,children_count,compliance_mappings,'
                'compliance_regulation,controls,coverage,covered,created_at,'
                'description,external_id,external_parent_id,identifier,'
                'in_scope_children_count,name,parent,position,rationale,'
                'related_controls,related_requirements'
               )
    if resource.lower()=='compliance_mapping':
        return (
                f'https://apis-us.diligentoneplatform.com/v1/orgs/{org_id}/compliance_mappings/{id}?'
                'fields[compliance_mappings]=coverage,created_at,updated_at,compliance_requirement,control'
               )
    if resource.lower()=='result':
        return (
                f'https://apis-us.diligentoneplatform.com/v1/orgs/{org_id}/tables/{id}/records'
               )
    raise Exception(f'the resource name {resource} is not found')
    

In [5]:
def fetch_data(resource, resource_id, max_retries=5, timeout=10, backoff_factor=1):
    """
        fetch data from endpoint with maximum retries, timeout and session management

        Args:
            resource(str): the highbond resource from which to retrieve data
            resource_id(str): identifier for the resource instance
            max_retries(int): no of retry attempt upon non-response from the server
            timeout(int): prevent prolonged server connection
            backoff_factor = wait multiplier between successive request
        Return:
            resource_data(list, dict): list/dict of objects containing data from the endpoint
    """
    retries = Retry(total= max_retries, 
                    backoff_factor=backoff_factor, 
                    status_forcelist=[429, 500, 502, 503, 504], #retries on these http errors
                    allowed_methods=['GET'])
    session = requests.session()
    adapter = HTTPAdapter(max_retries=retries)
    session.mount('http://',adapter)
    session.mount('https://', adapter)
    #all_data = []
    # get resource url 
    ids = resource_id if isinstance(resource_id, list) else [resource_id]
    for id  in ids:
        try:
            url = resource_url(resource, id)
            #print(id)
        except Exception as e:
            print(e)
        # fetched data for the specified resources
        while url:
            try:
                resp = session.get(url, headers=headers,timeout=timeout)
                resp.raise_for_status()
                #print(url)
            except Exception as e:
                print(f'❌ failed to fetch data: {e}')
            resource_data = resp.json().get('data', [])
            #print(resource_data)
            #all_data = all_data + project_data if isinstance(project_data, list) else project_data # check and accumulate multiple project or return a single targerted project
            yield resource_data
            next_page = resp.json().get('links',{}).get('next','') if isinstance(resource_data, list) else None
            url = urljoin(domain_url,next_page) if next_page else None
           
    
    session.close()
    

In [6]:
# get a list representation of all data fetched from a specific highbond resource

def get_data_collection(resource, resource_id=''):
    """ 
        Return a collection of all data retrieved from the highbond resource

        Args:
            resource(str): label describing the Highond resource from which to pull data
            resource_id(str): identifier for targeted HB resource
        Return:
            all_data(list): list of data objects fetched form the resource
    """
    all_data = []
    for data_obj in fetch_data(resource, resource_id):
        if isinstance(data_obj, list):
            all_data = all_data + data_obj  
        else:
            all_data.append(data_obj)
    return all_data

In [7]:
###########################################################################################
# fetch highbond data
###########################################################################################

# fetch project data
project_data = get_data_collection('project','492704')
project_id = [project['id'] for project in project_data]

# fetch objectives data
objectives = get_data_collection('objective',project_id)

# fetch risk data
objective_id = [objective['id'] for objective in objectives]
objective_id
risks = get_data_collection('risk', objective_id)

# import control data
controls = get_data_collection('control', objective_id)

# get control test data
control_tests = get_data_collection('control_test')

# import walthrough data 
walkthroughs = get_data_collection('walkthrough')

# import issues data 
findings = get_data_collection('issue')
org_issues = [issue for issue in findings if issue['relationships']['project']['data']['id']==project_id[0]] # filter org specific issues

#import action data 
issue_ids = [issue['id'] for issue in findings]
actions = get_data_collection('action', issue_ids)

# import results 
result_data = get_data_collection('result', '837936')

# import mitigation data 
# issue_ids = [issue['id'] for issue in control_tests_issues]
# mitigations = get_data_collection('mitigation',issue_ids[0])
# mitigations

# import regulation data
# regulations = get_data_collection('compliance_regulation')
# regulations

# import requirements data
# regulation_ids = [regulation['id'] for regulation in regulations]
# requirements = get_data_collection('compliance_requirement',regulation_ids[0])
# requirements

# import compliance mapping data 
# requirement_ids = [req['id'] for req in requirements]
# compliance_mapping = get_data_collection('compliance_mapping', requirement_ids)
# compliance_mapping

In [8]:
def transfrom_val(val, collapse=True):
    """
        Return suitable value and type for a given value

        Args:
            val(any): value of any type to be transformed
            collapse(Bool): specify whether or not a collection should be converted to string
        Return trans_val(any): transformed value
    """
    if isinstance(val, str):
        #return nan for blank values
        if not val:
            return None
        return val
    if isinstance(val, list):
        if collapse:
            return ','.join(val) if val else None
        return val if val else val.append(None)
    #return val[val]


In [9]:
def flatten_attributes(d):
    """ 
        Apply level 1 flattening to resource attributes and relationships

        Args:
            d(dict): dictionary object containing nested attributes to be flatten
        
        Return:
            dict_obj(dict): flattened distionary object
    """
    dict_obj = copy.deepcopy(d)
    #flatten attributes and custom attributes
    if 'attributes' in dict_obj:
        attrs = dict_obj['attributes'] 
        if 'custom_attributes' in attrs:
            cust_attrs = attrs['custom_attributes']
            del attrs['custom_attributes']
            for attr in cust_attrs:
                if isinstance(attr, dict):
                    # create new dict with custom attribute labels and values
                    new_dict = {}
                    term = attr['term']
                    val = transfrom_val(attr['value']) #if not isinstance(attr['value'],list or dict) else ','.join(attr['value'])
                    new_dict[term] = val
                    attrs.update(new_dict)
        del dict_obj['attributes']
        dict_obj.update(attrs)
    # flatten relationship attributes having list values
    if 'relationships' in dict_obj:
        relationships = dict_obj['relationships']
        relate_attrs = ['mitigations','control_tests']
        for r_attr in relate_attrs:
            if r_attr in relationships:
                relationships[f'{r_attr}_id'] = [attr.get('id') for attr in relationships[r_attr]['data']] \
                      if relationships[r_attr]['data'] else [None]
                del relationships[r_attr]
        dict_obj['relationships'] = relationships
    
    return dict_obj

In [10]:
def clean_html(val):
    """ 
        Return a string value with html tags and special characters removed
        Args:
            val(str): string value to be cleaned
        Return:
            clean_str(str): string values with html tags removed
    """
    decode = html.unescape(val) #convert html entites (lt;,gt;, nbsp;) to tag literals
    html_rem = re.sub(r'<.*?>',"", decode)
    rem_special = re.sub(r'[^a-zA-Z0-9\s.,!?-]', "", html_rem) # remove special characters
    clean_val = re.sub(r'\s+'," ", rem_special).strip() # noramilze whitespace
    return clean_val




In [11]:
def flatten_data(d, parent_key='', sep='_'):
    """ 
        Return a flattened dict object with nested dict removed

        Args:
            d(dict): dictionary containing nested data
            parent_key(str): holds key values for flatten dict object
            sep(str): char for combining flattened keys
        
        Return:
            items(dict): flattened dictionary with all nestings removed
    """
    items = {}
    d = flatten_attributes(d)
    for k,v in d.items():
        accum_keys = f'{parent_key}{sep}{k}' if parent_key else k
        if isinstance(v,dict): # check if value is a collection type-dict
            is_dict = False
            for val in v.values():
                if isinstance(val, dict):
                    is_dict = True
            if not is_dict and 'relationships' in accum_keys:
                keys = list(v.keys())
                vals = list(v.values())
                if 'framework' in accum_keys:
                    new_key = f'framework{sep}{vals[1]}{sep}{keys[0]}' # prefix framework relationships keys(val,key) with 'framework' label
                elif len(keys) <= 1:
                    attr_key = '_'.join(accum_keys.split('_')[1:]) #remove 'relationships' label from accum key
                    new_key = f'{attr_key}{sep}id' # relationship attributes without data
                else:
                    new_key = f'{vals[1]}{sep}{keys[0]}'
                items[new_key] = vals[0] if len(keys) > 1 else None # assign blanks to relationship attributes without data
            else:
                items.update(flatten_data(v,accum_keys,sep=sep))
        else:
            items[k] = transfrom_val(clean_html(v)) if isinstance(v,str) else v
    return items

In [12]:
# create dataframes for data

def get_source_frames(data, fields={}, record_path=''):
    """ 
        Return a dataframe for Highbond source data

        Args:
            data(list): list of dict object containing highbond data
            fields(list): list of attributes/fields to include in the df
            record_path(str): attribute for expanding the data
        Return:
            df(dataframe): dataframe of HB data
    """
    
    # flatten data dict
    clean_data = [flatten_data(item) for item in data]
    # subset required fileds
    if fields:
        # add type attribute to fields by default to enable checking for risk object for normailization purpose
        # if not fields.get('type'):
        #     fields['type'] = 'type'
        sub_data = []
        for item in clean_data:
            new_item = {}
            for k, v in item.items():
                if k in fields:
                    new_item[k] = v
            sub_data.append(new_item)
        df = pd.DataFrame(sub_data) if not record_path else \
            pd.json_normalize(sub_data, record_path=record_path, 
                              meta=[k for k in sub_data[0]if k != record_path],
                              record_prefix=record_path
                              ) # expand records for specified record path
        fields[f'{record_path}0'] = record_path # add field for expanded record to selected fields
        df_rename = df.rename(columns=fields)
        return df_rename
    return pd.DataFrame(clean_data) if not record_path else \
            pd.json_normalize(clean_data, record_path=record_path, 
                              meta=[k for k in clean_data[0] if k!=record_path],
                              record_prefix=record_path)

In [15]:
###################################################################################################
# create dataframes for imported org data
###################################################################################################
# create dataframes for org issues data
fields = {
            'reference':'ref',
            'title':'finding',
            'Site':'site',
            'description':'observation',
            'risk':'impact',
            'recommendation':'recommendation',
            'remediation_plan':'aggreed_action_plan',
            'owner':'owner',
            'remediation_date':'remediation_due_date',
            'control_tests_id':'control_tests_id',
            'walkthroughs_id': 'walkthroughs_id'    
}
df_findings = get_source_frames(org_issues, fields=fields) # '10357907', '10357906'
df_findings

,control_tests_id,finding,observation,owner,recommendation,ref,impact,aggreed_action_plan,remediation_due_date,site,walkthroughs_id
0,9882762,Lack of user manual or reference documents for...,Lack of user manuals or reference documents fo...,Manager Support,Technicians Support should keep user manuals a...,1,Not referencing preventive maintenance to user...,The missing manuals reference documents are to...,2023-11-30,Nairobi,NaN
1,9882765,Absence of job cards for some critical assets,Job cards for implementing support of some cri...,Manager,Introduction of condition monitoring checklist...,2,This may affect effective implementation of PM...,None,None,All Offices,NaN
2,9882766,Missing job cards for preventive maintenance,Missing job cards for preventive maintenance w...,Manager IT,Improvement on the filing Ensuring 100 adheren...,3,Non-adherence to PMP may affect the overall eq...,None,None,All Sites,NaN
3,9882767,Un-updated job cards and PMP,Mismatch between job cards and PMP in terms of...,IT manager,Review of the existing job cards with referenc...,4,Different PMP schedules for the same equipment...,None,None,All Site,NaN
4,9882768,Poor Structural condition on Foam tank and Die...,oor structural condition for some critical equ...,Manager IT,Corrective maintenance should be done on the a...,5,Ineffective response to an emergency because t...,None,None,All Site,NaN
5,9882763,Absence of test plan for satellites depots,"Satellite depots equipment not on PMP Mchinji,...",Manager IT,A PMP for these sites should be developed and ...,6,No guarantee can be placed on the integrity of...,None,None,Johannessburg,NaN
6,9882764,Failure to adhere to contractual terms Mchinji...,Nonadherence to contractual obligations on per...,IT Manager,Enforcement of the contractual obligations to ...,7,This will affect the overall equipment integrity,None,None,Nairobi,NaN
7,9882764,No preventive maintenance or condition monitor...,No preventive maintenance or condition monitor...,Manager IT,Developing condition monitoring checklist for ...,8,This will affect the overall equipment integrity,None,None,All Sites,NaN
8,NaN,T.Absence of job cards for some critical assets,Job cards for implementing support of some cri...,Manager,Introduction of condition monitoring checklist...,01,This may affect effective implementation of PM...,None,None,Joburg,10357907
9,9882768,T.Missing job cards for preventive maintenance,Missing job cards for preventive maintenance w...,Manager IT,Improvement on the filing Ensuring 100 adheren...,02,Non-adherence to PMP may affect the overall eq...,None,None,All Sites,NaN


In [16]:
# create dataframe for actions
# df_actions = get_source_frames(actions)
# df_actions.head()

In [17]:
# compute risk rating 
def risk_rating(risk):
    """ return risk rating"""
    rating = risk_matrix[risk.loc['impact']][risk.loc['likelihood']]['value']
    return rating

# create dataframe for risk
fields = {
            'mitigations_id':'mitigations_id',
            'risk_id':'risk_id',
            'description':'risk_description',
            'impact':'impact',
            'likelihood':'likelihood',
            'inherent_risk':'inherent_risk',
            'residual_risk':'residual_risk',
            'assurance':'assurance'
}
df_risks = get_source_frames(risks, fields=fields, record_path='mitigations_id')
df_risks['rating'] = df_risks.apply(risk_rating, axis=1) # derive risk rating for impact and likelihood values
df_risks.head()

,mitigations_id,risk_description,risk_id,impact,likelihood,inherent_risk,residual_risk,assurance,rating
0,14103230,Non-adherence to approved Preventive Maintenan...,R1,Major,Likely,16.0,12.8,20,VH(16)
1,14103231,Non-adherence to approved Preventive Maintenan...,R1,Major,Likely,16.0,12.8,20,VH(16)
2,14103232,Non-adherence to approved Preventive Maintenan...,R1,Major,Likely,16.0,12.8,20,VH(16)
3,14103233,Incompleteness of the PMP may result in reject...,R2,Major,Likely,16.0,16.0,0,VH(16)
4,14103234,Incompleteness of the PMP may result in reject...,R2,Major,Likely,16.0,16.0,0,VH(16)


In [18]:
# create control dataframe
fields = {
            'id':'id',
            'control_id':'control_id',
            'control_tests_id':'control_tests_id',
            'walkthroughs_id':'walkthroughs_id',
            'mitigations_id':'mitigations_id',
            'description':'control_description'
}
df_controls = get_source_frames(controls, fields=fields, record_path='control_tests_id') 
df_controls['mitigations_id'] = df_controls['mitigations_id'].apply(lambda x: ','.join(x)) 
df_controls

,control_tests_id,id,walkthroughs_id,mitigations_id,control_description,control_id
0,9882762,14421648,10357901,14103230,Training and experience of support is pivotal ...,01
1,9882763,14421649,10357902,14103231,Random inconsistent scheduled visits to these ...,06
2,9882764,14421650,10357903,14103232,Contractual Agreement,07
3,9882765,14421651,10357904,14103233,All sites have condition monitoring checklists...,02
4,9882766,14421652,10357905,14103234,External contractors carry out annual preventi...,03
5,9882767,14421653,10357906,14103235,Review of job cards by Manager Supervisor,04
6,9882768,14421654,10357907,14103236,Condition monitoring inspections and checklists,05


In [19]:
# create walkthrough dataframe

df_walkthrough = get_source_frames(walkthroughs)
df_walkthrough['opinion_per_risk'] = df_walkthrough.apply(lambda x: 'Designed Appropriately' if x['control_design'] else 'Inadequate design' 
                                                          if x['control_design']==False and not x is None else '', axis=1)
#df_walkthrough[df_walkthrough['controls_id']=='14421654' ]
df_walkthrough.head()

,id,type,controls_id,walkthrough_results,control_design,created_at,updated_at,Measurement of Effectiveness(Risk Indicator),DESIGN EFFECTIVENESS ASSESSMENT (DEA) QUESTIONS,DEA Questions (Select all questions),...,Have all DEA questions been answered?,Comments,Action Plan to further control this Risk,Are you only partially satisfied with the testing?,Control Design - Overall Conclusion,Insurance Applicable,Residual Exposure (Insurance Applicable),Residual Exposure ( Insurance Not Applicable),COBIT Adequacy Audit Programme,opinion_per_risk
0,575084,walkthroughs,570628,None,None,2015-02-17T105631Z,2015-02-17T105631Z,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
1,1924117,walkthroughs,2082545,MOUs not updated.,False,2017-10-08T144404Z,2017-10-08T162937Z,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Inadequate design
2,1924118,walkthroughs,2082546,None,None,2017-10-08T144710Z,2017-10-08T144710Z,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
3,1924119,walkthroughs,2082547,None,None,2017-10-08T145241Z,2017-10-08T145241Z,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
4,1924120,walkthroughs,2082548,None,None,2017-10-08T145559Z,2017-10-08T145559Z,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,


In [20]:
# create control tests dataframe
fields = {
            'id':'control_tests_id',
            'controls_id':'controls_id',
            'testing_conclusion_status':'testing_conclusion_status'
}
df_control_tests = get_source_frames(control_tests, fields=fields)
df_control_tests.head()
#df_control_tests[df_control_tests['id'] == '9882763']

,control_tests_id,controls_id,testing_conclusion_status
0,3561395,4884408,None
1,3561396,4884409,Operating Effectively
2,3561397,4884410,None
3,3605740,4947628,None
4,3605741,4947627,None


In [21]:
# combine controls, control test and walkthrough data 

df_control_tests_join = df_control_tests.merge(df_controls, how='inner', 
                                          on= 'control_tests_id')
df_control_tests_walkthrough = df_control_tests_join.merge(df_walkthrough[['id','controls_id','opinion_per_risk']], 
                                                         how='left', on='controls_id')
df_control_tests_walkthrough


,control_tests_id,controls_id,testing_conclusion_status,id_x,walkthroughs_id,mitigations_id,control_description,control_id,id_y,opinion_per_risk
0,9882762,14421648,Partially Operating Effectively,14421648,10357901,14103230,Training and experience of support is pivotal ...,01,10357901,Inadequate design
1,9882763,14421649,Operating Ineffectively,14421649,10357902,14103231,Random inconsistent scheduled visits to these ...,06,10357902,Designed Appropriately
2,9882764,14421650,Operating Effectively,14421650,10357903,14103232,Contractual Agreement,07,10357903,Designed Appropriately
3,9882765,14421651,Operating Ineffectively,14421651,10357904,14103233,All sites have condition monitoring checklists...,02,10357904,Designed Appropriately
4,9882766,14421652,Partially Operating Effectively,14421652,10357905,14103234,External contractors carry out annual preventi...,03,10357905,Designed Appropriately
5,9882767,14421653,Partially Operating Effectively,14421653,10357906,14103235,Review of job cards by Manager Supervisor,04,10357906,Inadequate design
6,9882768,14421654,Partially Operating Effectively,14421654,10357907,14103236,Condition monitoring inspections and checklists,05,10357907,Inadequate design


In [22]:
# combine risk and control data 
df_control_tests_walkthrough = df_control_tests_walkthrough[['control_tests_id','mitigations_id','walkthroughs_id','control_description',
                                                             'control_id','testing_conclusion_status',
                                                             'opinion_per_risk']]
df_risks_controls = df_risks.merge(df_control_tests_walkthrough, how='inner', on='mitigations_id')
df_risks_controls

,mitigations_id,risk_description,risk_id,impact,likelihood,inherent_risk,residual_risk,assurance,rating,control_tests_id,walkthroughs_id,control_description,control_id,testing_conclusion_status,opinion_per_risk
0,14103230,Non-adherence to approved Preventive Maintenan...,R1,Major,Likely,16.0,12.8,20,VH(16),9882762,10357901,Training and experience of support is pivotal ...,01,Partially Operating Effectively,Inadequate design
1,14103231,Non-adherence to approved Preventive Maintenan...,R1,Major,Likely,16.0,12.8,20,VH(16),9882763,10357902,Random inconsistent scheduled visits to these ...,06,Operating Ineffectively,Designed Appropriately
2,14103232,Non-adherence to approved Preventive Maintenan...,R1,Major,Likely,16.0,12.8,20,VH(16),9882764,10357903,Contractual Agreement,07,Operating Effectively,Designed Appropriately
3,14103233,Incompleteness of the PMP may result in reject...,R2,Major,Likely,16.0,16.0,0,VH(16),9882765,10357904,All sites have condition monitoring checklists...,02,Operating Ineffectively,Designed Appropriately
4,14103234,Incompleteness of the PMP may result in reject...,R2,Major,Likely,16.0,16.0,0,VH(16),9882766,10357905,External contractors carry out annual preventi...,03,Partially Operating Effectively,Designed Appropriately
5,14103235,Delays in purchase and delivery of maintenance...,R4,Catastrophic,Likely,20.0,20.0,0,VH(20i),9882767,10357906,Review of job cards by Manager Supervisor,04,Partially Operating Effectively,Inadequate design
6,14103236,Delays in purchase and delivery of maintenance...,R4,Catastrophic,Likely,20.0,20.0,0,VH(20i),9882768,10357907,Condition monitoring inspections and checklists,05,Partially Operating Effectively,Inadequate design


In [23]:
# combine issues, risk, control data 
df_test_issue_risk_control = df_findings.merge(df_risks_controls, how='inner', on='control_tests_id')
df_wk_issue_risk_control = df_findings.merge(df_risks_controls, how='inner', on='walkthroughs_id')
df_test_issue_risk_control

,control_tests_id,finding,observation,owner,recommendation,ref,impact_x,aggreed_action_plan,remediation_due_date,site,...,likelihood,inherent_risk,residual_risk,assurance,rating,walkthroughs_id_y,control_description,control_id,testing_conclusion_status,opinion_per_risk
0,9882762,Lack of user manual or reference documents for...,Lack of user manuals or reference documents fo...,Manager Support,Technicians Support should keep user manuals a...,1,Not referencing preventive maintenance to user...,The missing manuals reference documents are to...,2023-11-30,Nairobi,...,Likely,16.0,12.8,20,VH(16),10357901,Training and experience of support is pivotal ...,01,Partially Operating Effectively,Inadequate design
1,9882765,Absence of job cards for some critical assets,Job cards for implementing support of some cri...,Manager,Introduction of condition monitoring checklist...,2,This may affect effective implementation of PM...,None,None,All Offices,...,Likely,16.0,16.0,0,VH(16),10357904,All sites have condition monitoring checklists...,02,Operating Ineffectively,Designed Appropriately
2,9882766,Missing job cards for preventive maintenance,Missing job cards for preventive maintenance w...,Manager IT,Improvement on the filing Ensuring 100 adheren...,3,Non-adherence to PMP may affect the overall eq...,None,None,All Sites,...,Likely,16.0,16.0,0,VH(16),10357905,External contractors carry out annual preventi...,03,Partially Operating Effectively,Designed Appropriately
3,9882767,Un-updated job cards and PMP,Mismatch between job cards and PMP in terms of...,IT manager,Review of the existing job cards with referenc...,4,Different PMP schedules for the same equipment...,None,None,All Site,...,Likely,20.0,20.0,0,VH(20i),10357906,Review of job cards by Manager Supervisor,04,Partially Operating Effectively,Inadequate design
4,9882768,Poor Structural condition on Foam tank and Die...,oor structural condition for some critical equ...,Manager IT,Corrective maintenance should be done on the a...,5,Ineffective response to an emergency because t...,None,None,All Site,...,Likely,20.0,20.0,0,VH(20i),10357907,Condition monitoring inspections and checklists,05,Partially Operating Effectively,Inadequate design
5,9882763,Absence of test plan for satellites depots,"Satellite depots equipment not on PMP Mchinji,...",Manager IT,A PMP for these sites should be developed and ...,6,No guarantee can be placed on the integrity of...,None,None,Johannessburg,...,Likely,16.0,12.8,20,VH(16),10357902,Random inconsistent scheduled visits to these ...,06,Operating Ineffectively,Designed Appropriately
6,9882764,Failure to adhere to contractual terms Mchinji...,Nonadherence to contractual obligations on per...,IT Manager,Enforcement of the contractual obligations to ...,7,This will affect the overall equipment integrity,None,None,Nairobi,...,Likely,16.0,12.8,20,VH(16),10357903,Contractual Agreement,07,Operating Effectively,Designed Appropriately
7,9882764,No preventive maintenance or condition monitor...,No preventive maintenance or condition monitor...,Manager IT,Developing condition monitoring checklist for ...,8,This will affect the overall equipment integrity,None,None,All Sites,...,Likely,16.0,12.8,20,VH(16),10357903,Contractual Agreement,07,Operating Effectively,Designed Appropriately
8,9882768,T.Missing job cards for preventive maintenance,Missing job cards for preventive maintenance w...,Manager IT,Improvement on the filing Ensuring 100 adheren...,02,Non-adherence to PMP may affect the overall eq...,None,None,All Sites,...,Likely,20.0,20.0,0,VH(20i),10357907,Condition monitoring inspections and checklists,05,Partially Operating Effectively,Inadequate design
9,9882768,T.Un-updated job cards and PMP,Mismatch between job cards and PMP in terms of...,IT manager,Review of the existing job cards with referenc...,03,Different PMP schedules for the same equipment...,None,None,All Site,...,Likely,20.0,20.0,0,VH(20i),10357907,Condition monitoring inspections and checklists,05,Partiall

In [24]:
##########################################################################################
# subsets- prepare data for report
###########################################################################################

audit_result_fields = {
                        'risk_id':'risk_id',
                        'risk_description':'risk_description',
                        'rating':'rating',
                        'site':'site',
                        'finding':'finding',
                        'ref': 'ref',
                        'opinion_per_risk':'opinion_per_risk',
                        'impact_y':'impact',
                        'likelihood':'likelihood'
                      }
df_test_issue_risk_control = df_test_issue_risk_control[audit_result_fields.keys()].rename(columns=audit_result_fields)
df_wk_issue_risk_control = df_wk_issue_risk_control[audit_result_fields.keys()].rename(columns=audit_result_fields)
df_audit_result_summ = pd.concat([df_test_issue_risk_control, df_wk_issue_risk_control], ignore_index=True)
df_audit_result_summ

,risk_id,risk_description,rating,site,finding,ref,opinion_per_risk,impact,likelihood
0,R1,Non-adherence to approved Preventive Maintenan...,VH(16),Nairobi,Lack of user manual or reference documents for...,1,Inadequate design,Major,Likely
1,R2,Incompleteness of the PMP may result in reject...,VH(16),All Offices,Absence of job cards for some critical assets,2,Designed Appropriately,Major,Likely
2,R2,Incompleteness of the PMP may result in reject...,VH(16),All Sites,Missing job cards for preventive maintenance,3,Designed Appropriately,Major,Likely
3,R4,Delays in purchase and delivery of maintenance...,VH(20i),All Site,Un-updated job cards and PMP,4,Inadequate design,Catastrophic,Likely
4,R4,Delays in purchase and delivery of maintenance...,VH(20i),All Site,Poor Structural condition on Foam tank and Die...,5,Inadequate design,Catastrophic,Likely
5,R1,Non-adherence to approved Preventive Maintenan...,VH(16),Johannessburg,Absence of test plan for satellites depots,6,Designed Appropriately,Major,Likely
6,R1,Non-adherence to approved Preventive Maintenan...,VH(16),Nairobi,Failure to adhere to contractual terms Mchinji...,7,Designed Appropriately,Major,Likely
7,R1,Non-adherence to approved Preventive Maintenan...,VH(16),All Sites,No preventive maintenance or condition monitor...,8,Designed Appropriately,Major,Likely
8,R4,Delays in purchase and delivery of maintenance...,VH(20i),All Sites,T.Missing job cards for preventive maintenance,02,Inadequate design,Catastrophic,Likely
9,R4,Delays in purchase and delivery of maintenance...,VH(20i),All Site,T.Un-updated job cards and PMP,03,Inadequate design,Catastrophic,Likely


In [25]:
audit_details_fields = {
                        'ref':'ref',
                        'finding':'finding',
                        'site':'site',
                        'observation':'observation',
                        'impact_x':'impact',
                        'control_description':'other_existing_mitigation',
                        'recommendation':'recommendation',
                        'aggreed_action_plan':'aggreed_action_plan',
                        'responsible_personnel/due_date':'responsible_personnel/due_date'
                        }

def concate_attribute(d, x='owner', y='remediation_due_date'):
    attr1 = d[x] 
    attr2 = d[y] 
    #print(np.isnan(attr1), np.isnan(attr2))
    if attr1 and attr2:
        return attr1+' '+ attr2
    elif attr1:
        return attr1
    elif attr2:
        return attr2 
    else:
        None
df_issue_risk_control = df_findings.merge(df_risks_controls, how='left', on='control_tests_id')
df_issue_risk_control['responsible_personnel/due_date'] = df_issue_risk_control.apply(concate_attribute, axis=1)
df_detailed_findings = df_issue_risk_control[audit_details_fields.keys()].rename(columns=audit_details_fields)
df_detailed_findings

,ref,finding,site,observation,impact,other_existing_mitigation,recommendation,aggreed_action_plan,responsible_personnel/due_date
0,1,Lack of user manual or reference documents for...,Nairobi,Lack of user manuals or reference documents fo...,Not referencing preventive maintenance to user...,Training and experience of support is pivotal ...,Technicians Support should keep user manuals a...,The missing manuals reference documents are to...,Manager Support 2023-11-30
1,2,Absence of job cards for some critical assets,All Offices,Job cards for implementing support of some cri...,This may affect effective implementation of PM...,All sites have condition monitoring checklists...,Introduction of condition monitoring checklist...,None,Manager
2,3,Missing job cards for preventive maintenance,All Sites,Missing job cards for preventive maintenance w...,Non-adherence to PMP may affect the overall eq...,External contractors carry out annual preventi...,Improvement on the filing Ensuring 100 adheren...,None,Manager IT
3,4,Un-updated job cards and PMP,All Site,Mismatch between job cards and PMP in terms of...,Different PMP schedules for the same equipment...,Review of job cards by Manager Supervisor,Review of the existing job cards with referenc...,None,IT manager
4,5,Poor Structural condition on Foam tank and Die...,All Site,oor structural condition for some critical equ...,Ineffective response to an emergency because t...,Condition monitoring inspections and checklists,Corrective maintenance should be done on the a...,None,Manager IT
5,6,Absence of test plan for satellites depots,Johannessburg,"Satellite depots equipment not on PMP Mchinji,...",No guarantee can be placed on the integrity of...,Random inconsistent scheduled visits to these ...,A PMP for these sites should be developed and ...,None,Manager IT
6,7,Failure to adhere to contractual terms Mchinji...,Nairobi,Nonadherence to contractual obligations on per...,This will affect the overall equipment integrity,Contractual Agreement,Enforcement of the contractual obligations to ...,None,IT Manager
7,8,No preventive maintenance or condition monitor...,All Sites,No preventive maintenance or condition monitor...,This will affect the overall equipment integrity,Contractual Agreement,Developing condition monitoring checklist for ...,None,Manager IT
8,01,T.Absence of job cards for some critical assets,Joburg,Job cards for implementing support of some cri...,This may affect effective implementation of PM...,NaN,Introduction of condition monitoring checklist...,None,Manager
9,02,T.Missing job cards for preventive maintenance,All Sites,Missing job cards for preventive maintenance w...,Non-adherence to PMP may affect the overall eq...,Condition monitoring inspections and checklists,Improvement on the filing Ensuring 100 adheren...,None,Manager IT


# re-structure data for report generation
################################################################################

# project data 
project = flatten_data(project_data[0])
## format and rename report date attribute
project['report_date'] = dt.strptime(project['Report Date'], '%Y-%m-%d').strftime('%B %Y')
del project['Report Date']


# risk, finding, control data combinned

risk_findings = df_audit_result_summ.to_dict(orient='records')
risk_ids = set([item.get('risk_id') for item in risk_findings]) #get unique risk id
# get risk and related findings attributes
risk_attrs = ['risk_id','risk_description','rating','impact','likelihood']
findings_attrs = [attr for attr in df_audit_result_summ.columns if attr not in risk_attrs]
# aggregated all findings for each risk into a single object (dict)
new_risk_findings = []
for risk_id in risk_ids:
    new_risk_find={}
    for rec in risk_findings:
        finding_rec = {} # dict for extracting findings data
        if rec.get('risk_id')==risk_id:  
            if 'risk_id' not in new_risk_find: # extract risk data once per risk.
                for risk_attr in risk_attrs:
                    new_risk_find[risk_attr] = rec.get(risk_attr) if rec.get(risk_attr) else ''
            # extract and add findings to a list for a specific risk_id
            for findings_attr in findings_attrs: 
                if 'finding_attributes' not in new_risk_find:
                    new_risk_find['finding_attributes']=[]
                finding_rec[findings_attr] = '' if not rec.get(findings_attr) or rec.get(findings_attr)!= rec.get(findings_attr) \
                      else rec.get(findings_attr) # return empty string for None and Nan types (nan val is not equal to itself)
            new_risk_find['finding_attributes'].append(finding_rec)
    new_risk_findings.append(new_risk_find)
# sort risk
risk_findings_sorted = sorted(new_risk_findings, key=lambda x: x['risk_id'])
risk_findings_sorted

In [26]:
# re-structure data for report generation2
################################################################################

# project data 
project = flatten_data(project_data[0])
project['type_name'] = project['Report Type'].upper()+'-'+project['name']
project['report_type'] = project['Report Type']
## format and rename report date attribute
project['report_date'] = dt.strptime(project['Report Date'], '%Y-%m-%d').strftime('%B %Y')
del project['Report Date']


## group findings data by risk and site
#*******************************************************************************************

# risk, finding, control data combinned

risk_findings = df_audit_result_summ.to_dict(orient='records')
risk_ids = set([item.get('risk_id') for item in risk_findings]) #get unique risk id
# get risk and related findings attributes from risk_find
risk_attrs = ['risk_id','risk_description','rating','impact','likelihood']
findings_attrs = [attr for attr in df_audit_result_summ.columns if attr not in risk_attrs]
# aggregated all findings for each risk into a single object (dict)
new_risk_findings = []
for risk_id in risk_ids:
    new_risk_find={}
    for rec in risk_findings:
        finding_rec = {} # dict for extracting findings data
        if rec.get('risk_id')==risk_id:  
            if 'risk_id' not in new_risk_find: # extract risk data once per risk.
                for risk_attr in risk_attrs:
                    new_risk_find[risk_attr] = rec.get(risk_attr) if rec.get(risk_attr) else ''
            # extract and add findings to a list for a specific risk_id and site
            if 'sites' not in new_risk_find:
                    new_risk_find['sites']={}
            # set site key and aggregate findings per site
            site_name = rec['site'] 
            if not new_risk_find.get('sites',{}).get(site_name):
                new_risk_find['sites'][site_name] = []        
            for findings_attr in findings_attrs: 
                finding_rec[findings_attr] = '' if not rec.get(findings_attr) or rec.get(findings_attr)!= rec.get(findings_attr) \
                      else rec.get(findings_attr) # return empty string for None and Nan types (nan val is not equal to itself)
            new_risk_find['sites'][site_name].append(finding_rec)
    new_risk_findings.append(new_risk_find)
# sort risk
risk_findings_sorted = sorted(new_risk_findings, key=lambda x: x['risk_id'])
risk_findings_sorted

[{'risk_id': 'R1',
  'risk_description': 'Non-adherence to approved Preventive Maintenance Plan PMP of critical assets that might compromise the safety of the operations',
  'rating': 'VH(16)',
  'impact': 'Major',
  'likelihood': 'Likely',
  'sites': {'Nairobi': [{'site': 'Nairobi',
     'finding': 'Lack of user manual or reference documents for some critical equipment',
     'ref': '1',
     'opinion_per_risk': 'Inadequate design'},
    {'site': 'Nairobi',
     'finding': 'Failure to adhere to contractual terms Mchinji Concession Agreement',
     'ref': '7',
     'opinion_per_risk': 'Designed Appropriately'}],
   'Johannessburg': [{'site': 'Johannessburg',
     'finding': 'Absence of test plan for satellites depots',
     'ref': '6',
     'opinion_per_risk': 'Designed Appropriately'}],
   'All Sites': [{'site': 'All Sites',
     'finding': 'No preventive maintenance or condition monitoring of some equipment',
     'ref': '8',
     'opinion_per_risk': 'Designed Appropriately'}]}},
 {'

In [27]:
# finding details
detailed_findings = df_detailed_findings.to_dict(orient='records')
detailed_findings_sorted = sorted(detailed_findings, key=lambda x:x['ref'])
detailed_findings_sorted 

[{'ref': '01',
  'finding': 'T.Absence of job cards for some critical assets',
  'site': 'Joburg',
  'observation': 'Job cards for implementing support of some critical assets are yet to be developed. This is due to the maturity level of Preventive Maintenance management system as this process has just been initiated. Examples include Foam tank operating valves Pressure Vacuum vents Thermal relief valves Offloading metering system Tank valves inlet, outlet, drain and Water tanks.',
  'impact': 'This may affect effective implementation of PMP and eventually affecting the overall equipment integrity',
  'other_existing_mitigation': nan,
  'recommendation': 'Introduction of condition monitoring checklists for some of the equipment by bundling related equipment and accessories in one checklist or job card for example checklist for waterfoam fire suppression system and accessories',
  'aggreed_action_plan': None,
  'responsible_personnel/due_date': 'Manager'},
 {'ref': '02',
  'finding': 'T

In [28]:
## *************************************************************************************************
# Table Utilities
## ************************************************************************************************

def adjust_column_width(table, section):
    """
        Adjust the width of columns in a table based column header and text length

        Args:
            section(Section): document session object to contain the table
            table(table): table object with columns to be adjusted
        Return:
            None
    """
    # configure width and text wrapping for columns
    page_width = section.page_width - section.right_margin - section.left_margin  # avialable page width excluding margin
    col_text_lengths = []

    # average text length per column
    for i in range(len(table.columns)):
        sum_text_len = 0
        count = 0
        header_len = len(table.columns[i].cells[0].text.strip())
        for cell in table.columns[i].cells[1:]:
            cell_text = cell.text.strip()
            if cell_text:
                sum_text_len += len(cell.text.strip()) #len(table.cell(j,i).text.strip())
                count += 1
        avg_text_len = sum_text_len/count
        col_text_lengths.append((header_len,avg_text_len))

    col_text_len2 = []
    for h_len, t_len in col_text_lengths:
        # compare len of header and average len of column text to determine approximate col len
        if h_len >= t_len:
            col_len = h_len * 1.8
        else:
            len_ratio = h_len/t_len
            if len_ratio > 0.5:
                if len_ratio >= 0.9:
                    col_len = h_len * 1.5
                else:
                    col_len = h_len * 1.2
            else:
                len_ratio = h_len/t_len
                if len_ratio < 0.13:
                    col_len = max(h_len * 2, t_len * 0.01)
                else:
                    col_len = max(h_len * 3, t_len * 0.5)
        col_text_len2.append(col_len)

    total_len = sum(col_text_len2)
    width_ratios = [l/total_len for l in col_text_len2]  # [0.1,0.25,0.1,0.15,0.2,0.05,0.15]
    cols_width = [ratio*page_width for ratio in width_ratios]
    #apply proportional column widths 
    for x, col_width in enumerate(cols_width):
        #col_width = ratio * page_width
        for cell in table.columns[x].cells:
            cell.width = col_width

            # --- Add text wrapping XML (Word-level) ---
            # tcPr = cell._element.get_or_add_tcPr()
            # tcW = OxmlElement('w:tcW')
            # tcW.set(qn('w:w'), str(int(col_width)))   # width in twips
            # tcW.set(qn('w:type'), 'dxa')
            # tcPr.append(tcW)

            # Enable text wrapping and vertical alignment
            # tcPr.append(OxmlElement('w:noWrap'))  # explicitly enable wrapping
            # cell.vertical_alignment = 1  # top alignment

    #         # Allow line breaks inside the paragraph
    #         for paragraph in cell.paragraphs:
    #             for run in paragraph.runs:
    #                 run._element.get_or_add_rPr()

def adjust_column_width2(table, section):
    """
        Adjust the width of columns in a table based column header and text length

        Args:
            section(Section): document session object to contain the table
            table(table): table object with columns to be adjusted
        Return:
            None
    """
    # configure width and text wrapping for columns
    page_width = section.page_width - section.right_margin - section.left_margin  # avialable page width excluding margin
    col_text_lengths = [len(cell.text.strip()) for cell in table.rows[0].cells]
    
    # compute approximate lenght and rations
    
    col_text_len2 = []
    for col_len in col_text_lengths:
        if col_len < 15:
            if col_len < 7:
                col_len *= 1.8
            else:
                col_len *= 1.2
        else:
            if col_len > 25:
                col_len *= 0.4
            else:
                col_len *= 0.7
        col_text_len2.append(col_len)
    
    total_len = sum(col_text_len2)
    width_ratios = [l/total_len for l in col_text_len2]  # [0.1,0.25,0.1,0.15,0.2,0.05,0.15]
    cols_width = [ratio*page_width for ratio in width_ratios]

    #print(col_text_len2,width_ratios, cols_width)
    #apply proportional column widths 
    for x, col_width in enumerate(cols_width):
        #col_width = ratio * page_width
        for cell in table.columns[x].cells:
            cell.width = col_width

def add_border(table):
    """ add border to table"""
    for row in table.rows:
        for cell in row.cells:
            tcPr = cell._tc.get_or_add_tcPr()
            borders = OxmlElement('w:tcBorders')
            for edge in ('top', 'left', 'bottom', 'right'):
                border = OxmlElement(f'w:{edge}')
                border.set(qn('w:val'), 'single')
                border.set(qn('w:sz'), '4')
                border.set(qn('w:space'), '0')
                border.set(qn('w:color'), '000000')
                borders.append(border)
            tcPr.append(borders)

def repeat_tbl_header(table):
    """ Repeat table header at the start of a new page for table spanning multiple pages"""
    tr = table.rows[0]._tr
    trPr = tr.get_or_add_trPr()
    tblHeader = OxmlElement('w:tblHeader')
    tblHeader.set(qn('w:val'), "true")
    trPr.append(tblHeader)

def set_cell_bg_color(cell, color):
      """Change the bg color of table cells to the specified color"""
      tcPr = cell._tc.get_or_add_tcPr()
      shd_elm = parse_xml(r'<w:shd {} w:fill="{}"/>'.format(nsdecls('w'),color))
      tcPr.append(shd_elm)

def create_tbl_with_header(doc, columns, hd_bgcolor='D3D3D3'):
    """ 
        Return a table object object with formatted column headers

        Args:
            doc(documen/subdoc): doc or subdoc object to add the table
            columns(list): list of column header labels
            hd_bgcolor(str): hexcode for table header background color
        Return:
            table(table): table object with formatted column titles
    """

    table = doc.add_table(rows=1, cols=len(columns))

    table.autofit = False
    table.allow_autofit = False

    hdr = table.rows[0].cells

    # add header row
    for i, col in enumerate(columns):
        hdr[i].text = col
        set_cell_bg_color(hdr[i], hd_bgcolor)
        hdr[i].paragraphs[0].runs[0].bold = True

    return table

def disable_row_spliting(tbl_row):
    """Prevent table row from spliting accross multiple page"""

    tr = tbl_row._tr
    trPr = tr.get_or_add_trPr()
    cant_split = OxmlElement('w:cantSplit')
    trPr.append(cant_split)

def merge_tbl_rows(table, cols_index, row_start, row_end):
    """ 
        Merge all table row between specified start and end indexes

        Args:
            table(table): table object containing rows to be merged
            cols_index(list): list of column indexes to merge rows
            row_start(int): index of row to start merging
            row_end(int): index of row to end merging
        Return:
            None
    """
    
    table_rows = table.rows[row_start:row_end]
    cols_index = cols_index if isinstance(cols_index, list) else [cols_index]
    if len(table_rows) > 1:
        for col_idx in cols_index:
            table_rows[0].cells[col_idx].merge(table_rows[-1].cells[col_idx])

def set_table_borders(table):
    tbl = table._tbl  # access underlying XML element
    tblPr = tbl.tblPr

    # Remove existing tblBorders if any
    for element in tblPr.findall(qn('w:tblBorders')):
        tblPr.remove(element)

    # Create new tblBorders element
    borders = OxmlElement('w:tblBorders')
    for border_name in ['top', 'left', 'bottom', 'right', 'insideH', 'insideV']:
        border_el = OxmlElement(f'w:{border_name}')
        border_el.set(qn('w:val'), 'single')
        border_el.set(qn('w:sz'), '4')
        border_el.set(qn('w:space'), '0')
        border_el.set(qn('w:color'), '000000')  # black
        borders.append(border_el)
    tblPr.append(borders)

def remove_trailing_paragraph(subdoc):
    """Clean trailing section properties (and empty paragraph if any)"""
    
    body = subdoc._element.body
    if body is not None and len(body) > 0:
        last = body[-1]
        if last.tag.endswith('sectPr'):
            body.remove(last)
        elif last.tag.endswith('p') and not list(last):
            body.remove(last)

def format_labels(val):
    """Return a label with each word capitalised"""
    new_val = []
    attrs = val.split('_')
    for attr in attrs:
            new_val.append(attr.capitalize())
    return ' '.join(new_val)

def extract_html_table(html_text):
    """Return list of dictionary object of data extract from a html table

       Args:
            html_text(str): string value containing html table
       Return:
            table_dict(list): list of dictionary obj
    """

    soup= BeautifulSoup(html_text, 'html.parser')
    table= soup.find('table')
    rows = table.find_all('tr')

    # get table header
    headers = [th.get_text(strip=True) for th in rows[0].find_all(['th','td'])]
    # get table rows
    data = []
    for row in rows[1:]:
        row_data = [th.get_text(strip=True) for th in row.find_all(['th','td'])]
        data.append(dict(zip(headers,row_data)))
    
    # get table header color
    th = rows[0].find(['th','td'])
    bg_color = [part.split(':')[-1].strip() for part in th.get('style','').split(';') if 'background-color' in part][0]
    
    return data, bg_color[1:]

def add_blank_lines(document, count=1):
    """
    Add a given number of blank lines (empty paragraphs) to a Word document.

    Parameters
    ----------
    document : docx.document.Document
        The target document where blank lines will be added.
    count : int
        Number of blank lines to insert (default is 1).
    """
    for _ in range(count):
        document.add_paragraph()



In [ ]:
#########################################################################################################
# generate report document
#########################################################################################################

tpl = DocxTemplate('test_tpl.docx')

## Audit team table
#******************************************************************************

team_doc = Subdoc(tpl)

# add audit team title
audit_p = team_doc.add_paragraph('Audit Team')
#audit_p.style = "Heading 1"
p_runs = audit_p.runs[0]
p_runs.bold = True

### add table to document
audit_team_data, hd_bgcolor = extract_html_table(flatten_attributes(project_data[0])['Audit Team'])
columns = [col for col in audit_team_data[0].keys()]
audit_columns = list(map(format_labels, columns))
audit_table = create_tbl_with_header(team_doc, audit_columns, hd_bgcolor)

### set table header row to repeat across multiple pages
repeat_tbl_header(audit_table)

### add rows to findings table
for person in audit_team_data:
    row = audit_table.add_row()
    cells = row.cells
    for i, col in enumerate(columns):
        cells[i].text = person[col] if person[col] and person[col]==person[col] else ''
        #cells[i].paragraphs[0].runs[0].font.size = Pt(10)
    
    # prevent row spliting across multiple pages 
    disable_row_spliting(row)

#adjust_column_width2(audit_table, tpl.sections[1])
set_table_borders(audit_table)

add_blank_lines(team_doc, 1) # add blank line(s) between tables

## distribution list table 
#***********************************************************************************

# add distribution list title
dist_p = team_doc.add_paragraph('Distribution List')
#dist_p.style = "Heading 1"
p_runs = dist_p.runs[0]
p_runs.bold = True

distribution_list, hd_bgcolor = extract_html_table(flatten_attributes(project_data[0])['Distribution list'])
columns = [col for col in distribution_list[0].keys()]
dist_columns = list(map(format_labels, columns))
dist_table = create_tbl_with_header(team_doc, dist_columns, hd_bgcolor)

### set table header row to repeat across multiple pages
repeat_tbl_header(dist_table)

### add rows to findings table
for person in distribution_list:
    row = dist_table.add_row()
    cells = row.cells
    for i, col in enumerate(columns):
        cells[i].text = person[col] if person[col] and person[col]==person[col] else ''
        #cells[i].paragraphs[0].runs[0].font.size = Pt(10)
    
    # prevent row spliting across multiple pages 
    disable_row_spliting(row)

#adjust_column_width2(dist_table, tpl.sections[1])
set_table_borders(dist_table)


remove_trailing_paragraph(team_doc)


# generate risk summary table ************************************************************************

     
doc1 = Subdoc(tpl)
# add table to document
columns = list(map(format_labels, [col for col in df_audit_result_summ.columns if col not in ['impact','likelihood']]))
risk_table = create_tbl_with_header(doc1, columns)

# set table header row to repeat across multiple pages
repeat_tbl_header(risk_table)

# add data rows to table
for risk in risk_findings_sorted:
      start_of_risk = len(risk_table.rows)
      is_first_row = True
      for finding in risk.get('finding_attributes',[]):
            row = risk_table.add_row()
            cells = row.cells
            cells[0].text = risk.get('risk_id') if is_first_row else ''
            cells[1].text = risk.get('risk_description') if is_first_row else ''
            cells[2].text = risk.get('rating') if is_first_row else ''
            set_cell_bg_color(cells[2], risk_matrix.get(risk['impact'],{}).get(risk['likelihood'],{}).get('color')) # set bg color for risk rating
            cells[3].text = finding.get('site') 
            cells[4].text = finding.get('finding') 
            cells[5].text = finding.get('ref') 
            cells[6].text = finding.get('opinion_per_risk')

            # set background color for opinion per risk
            opinion_per_risk = finding.get('opinion_per_risk').lower() #if finding.get('opinion_per_risk') else ''
            set_cell_bg_color(cells[6], 'FF0000') if 'inadequate' in opinion_per_risk else \
                  set_cell_bg_color(cells[6], '00FF00') if 'appropriate' in opinion_per_risk else \
                        set_cell_bg_color(cells[6], 'FFFFFF')
            
            # prevent row spliting across multiple pages 
            disable_row_spliting(row)
            
            is_first_row = False
            
      # merge risk rows for a specific risk-finding combinations
      col_index = [0,1,2]  # columns to merger rows
      merge_tbl_rows(risk_table, col_index, start_of_risk, len(risk_table.rows))
      

adjust_column_width(risk_table, tpl.sections[2])
set_table_borders(risk_table)
remove_trailing_paragraph(doc1)

In [30]:
# generate risk finding summary table for risk and site grouping
doc1 = Subdoc(tpl)

# generate risk summary table
     

# add table to document
columns = list(map(format_labels, [col for col in df_audit_result_summ.columns if col not in ['impact','likelihood']]))
risk_table = create_tbl_with_header(doc1, columns)

# set table header row to repeat across multiple pages
repeat_tbl_header(risk_table)

# add data rows to table
for risk in risk_findings_sorted:
      start_of_risk = len(risk_table.rows)
      is_first_row = True
      for site in risk.get('sites',{}):
            start_of_site = len(risk_table.rows)
            is_first_row_site = True
            for finding in risk.get('sites',{}).get(site,[]):
                row = risk_table.add_row()
                cells = row.cells
                cells[0].text = risk.get('risk_id') if is_first_row else ''
                cells[1].text = risk.get('risk_description') if is_first_row else ''
                cells[2].text = risk.get('rating') if is_first_row else ''
                set_cell_bg_color(cells[2], risk_matrix.get(risk['impact'],{}).get(risk['likelihood'],{}).get('color')) # set bg color for risk rating
                cells[3].text = finding.get('site') if is_first_row_site else ''
                cells[4].text = finding.get('finding') 
                cells[5].text = finding.get('ref') 
                cells[6].text = finding.get('opinion_per_risk')

                # set background color for opinion per risk
                opinion_per_risk = finding.get('opinion_per_risk').lower() #if finding.get('opinion_per_risk') else ''
                set_cell_bg_color(cells[6], 'FF0000') if 'inadequate' in opinion_per_risk else \
                    set_cell_bg_color(cells[6], '00FF00') if 'appropriate' in opinion_per_risk else \
                            set_cell_bg_color(cells[6], 'FFFFFF')
                
                # prevent row spliting across multiple pages 
                disable_row_spliting(row)
                
                is_first_row = False
                is_first_row_site = False
            # merge site rows for a specific site-finding combinations
            col_index = 3  # column(s) to merger rows
            merge_tbl_rows(risk_table, col_index, start_of_site, len(risk_table.rows))

      # merge risk rows for a specific risk-finding combinations
      col_index = [0,1,2]  # columns to merger rows
      merge_tbl_rows(risk_table, col_index, start_of_risk, len(risk_table.rows))
      

adjust_column_width(risk_table, tpl.sections[2])
set_table_borders(risk_table)
remove_trailing_paragraph(doc1)

In [31]:
# create detailed findings table

doc2 = Subdoc(tpl)

# add table to document
columns = [col for col in detailed_findings[0].keys()]
findings_columns = list(map(format_labels, columns))
findings_table = create_tbl_with_header(doc2, findings_columns)

#set table header row to repeat across multiple pages
repeat_tbl_header(findings_table)

# add rows to findings table
for finding in detailed_findings_sorted:
    row = findings_table.add_row()
    cells = row.cells
    for i, col in enumerate(columns):
        cells[i].text = finding[col] if finding[col] and finding[col]==finding[col] else ''
        cells[i].paragraphs[0].runs[0].font.size = Pt(10)
    
    # prevent row spliting across multiple pages 
    disable_row_spliting(row)

adjust_column_width2(findings_table, tpl.sections[3])
set_table_borders(findings_table)
remove_trailing_paragraph(doc2)

In [32]:
# generate result table

columns = ['CUSTNO', 'AMOUNT', 'COUNT', 'COUNT PERCENTAGE', 'FIELD PERCENTAGE']

doc3 = Subdoc(tpl)

# add table to document
result_columns = list(map(format_labels, columns))
result_table = create_tbl_with_header(doc3, result_columns)

#set table header row to repeat across multiple pages
repeat_tbl_header(result_table)

# add rows to findings table
for rec in result_data:
    row = result_table.add_row()
    cells = row.cells
    for i, col in enumerate(columns):
        cells[i].text = rec[col] if rec[col] else ''
        #cells[i].paragraphs[0].runs[0].font.size = Pt(10)
    
    # prevent row spliting across multiple pages 
    disable_row_spliting(row)

adjust_column_width(result_table, tpl.sections[4])
set_table_borders(result_table)
remove_trailing_paragraph(doc3)

In [33]:
context = {
            'project': project,
            'auditsubdoc': team_doc,
            'mysubdoc1': doc1,
            'mysubdoc2': doc2,
            'mysubdoc3': doc3
}

tpl.render(context)
tpl.save('impact_report.docx')